In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 입력 폴더와 결과 폴더 경로 설정
input_folder = r"C:/temp2"  # 분석할 이미지 파일이 있는 폴더(원래 편석도측정 폴더에 있었는데 연결이 안되어서 폴더 변경, 한글 때문인듯.)
result_folder = r"C:/python_Study/편석도측정/Result"  # 분석 결과를 저장할 폴더
#C:\python_Study\편석도측정\pic 
# 결과 폴더가 없으면 생성
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

# 입력 폴더에서 .jpg 파일들만 선택
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

# 각 파일에 대해 분석 및 결과 저장
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    
    # 1. 이미지 읽기
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Could not load image {image_file}")
        continue

    # 2. 이미지 전처리 (블러링)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # 3. 이진화 처리 (Otsu's Method)
    _, binary_image = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # 4. 외곽선(Contours) 검출
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 5. 면적 계산
    total_area = image.shape[0] * image.shape[1]
    segregated_area = sum(cv2.contourArea(cnt) for cnt in contours)
    segregation_ratio = (segregated_area / total_area) * 100

    # 6. 결과 출력
    print(f"Image: {image_file}, Segregated Area: {segregated_area:.2f} pixels, Segregation Ratio: {segregation_ratio:.2f}%")

    # 7. 결과 이미지 시각화 (원본 이미지에 편석 영역 표시)
    result_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(result_image, contours, -1, (0, 0, 255), 2)

    # 8. 이진화 결과 및 히스토그램, 면적 등의 정보를 하나의 보고서로 결합하여 저장
    plt.figure(figsize=(12, 8))

    # 1. 원본 이미지
    plt.subplot(2, 2, 1)
    plt.title(f"Original Image - {image_file}")
    plt.imshow(image, cmap='gray')
    plt.axis('off')

    # 2. 이진화 결과
    plt.subplot(2, 2, 2)
    plt.title(f"Binary Image (After Otsu) - {image_file}")
    plt.imshow(binary_image, cmap='gray')
    plt.axis('off')

    # 3. 히스토그램
    plt.subplot(2, 2, 3)
    plt.hist(blurred.ravel(), bins=256, range=(0, 256))
    plt.title("Histogram of Blurred Image")
    plt.xlabel("Pixel Value")
    plt.ylabel("Frequency")


    # 4. 편석 영역 강조 (결과 이미지)
    plt.subplot(2, 2, 4)
    plt.title(f"Segregation Highlighted - {image_file}")
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # 9. 레이아웃 조정
    plt.subplots_adjust(left=0.05, right=0.95, top=0.85, bottom=0.15)  # 여백 조정
    
    # 히스토그램 아래에 텍스트 배치
    plt.figtext(0.5, 0.03, f"Total Area: {total_area} pixels | Segregated Area: {segregated_area:.2f} pixels | Segregation Ratio: {segregation_ratio:.2f}%",
                wrap=True, horizontalalignment='center', fontsize=12)

    # 보고서 이미지 저장
    result_report_path = os.path.join(result_folder, f"report_{image_file}.png")
    plt.tight_layout()
    plt.savefig(result_report_path)
    plt.close()

print("All files processed and reports saved.")


Image: longi1.jpg, Segregated Area: 1563881.00 pixels, Segregation Ratio: 48.29%
Image: longi2.jpg, Segregated Area: 1871675.00 pixels, Segregation Ratio: 60.76%
Image: longi3.jpg, Segregated Area: 1263734.50 pixels, Segregation Ratio: 38.83%
Image: longi4.jpg, Segregated Area: 1082115.00 pixels, Segregation Ratio: 44.69%
Image: t1.jpg, Segregated Area: 82259.00 pixels, Segregation Ratio: 43.80%
Image: t2.jpg, Segregated Area: 106486.50 pixels, Segregation Ratio: 29.84%
All files processed and reports saved.
